In [ ]:
import img_group
import tensorflow as tf
import numpy as np

batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w, strides=[1,1,1,1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1,1,1,1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    pyx = tf.matmul(l4, w_o)
    return pyx

result = img_group.get_data()
trX = np.array(result['train_images'])
trY = np.array(result['train_labels'])
teX = np.array(result['test_images'])
teY = np.array(result['test_labels'])

trX = trX.reshape(-1, 100, 100, 3)
teX = teX.reshape(-1, 100, 100, 3)

X = tf.placeholder('float', [None, 100, 100, 3])
Y = tf.placeholder('float', [None, 23])

w = init_weights([3, 3, 3, 32])
w2 = init_weights([3, 3, 32, 64])
w3 = init_weights([3, 3, 64, 128])
w4 = init_weights([128 * 13 * 13, 625])
w_o = init_weights([625, 23])

p_keep_conv = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')

py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.0009, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in xrange(1):
        training_batch = zip(xrange(0, len(trX), batch_size), xrange(batch_size, len(trX), batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], p_keep_conv:0.8, p_keep_hidden:0.5})
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        print (i, np.mean(np.argmax(teY[test_indices], axis=1) == 
                          sess.run(predict_op, feed_dict={X:teX[test_indices],
                                                          p_keep_conv:1.0,
                                                          p_keep_hidden:1.0})))

In [1]:
import img_group
import tensorflow as tf
import numpy as np
import time

batch_size = 128
test_size = 256
width = 40
height = 40

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    
    # l1a shape = (?, 40, 40, 32)
    # l1 shpae = (?, 20, 20, 32)
    biases = tf.get_variable('biases1', [32], initializer=tf.constant_initializer(0.0))
    conv1 = tf.nn.conv2d(X, w, strides=[1,1,1,1], padding='SAME')
    bias = tf.nn.bias_add(conv1, biases)
    l1a = tf.nn.relu(bias)
    l1 = tf.nn.max_pool(l1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    # l2a shape = (?, 20, 20, 64)
    # l2 shpae = (?, 10, 10, 64)
    biases = tf.get_variable('biases2', [64], initializer=tf.constant_initializer(0.1))
    conv2 = tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding='SAME')
    bias = tf.nn.bias_add(conv2, biases)
    l2a = tf.nn.relu(bias)
    l2 = tf.nn.max_pool(l2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    # l3a shape = (?, 10, 10, 128)
    # l3 shpae = (?, 5, 5, 128)
    # l3 reshape to (?, 128*5*5)
    biases = tf.get_variable('biases3', [128], initializer=tf.constant_initializer(0.1))
    conv3 = tf.nn.conv2d(l2, w3, strides=[1,1,1,1], padding='SAME')
    bias = tf.nn.bias_add(conv3, biases)
    l3a = tf.nn.relu(bias)
    l3 = tf.nn.max_pool(l3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    biases = tf.get_variable('biases4', [625], initializer=tf.constant_initializer(0.1))
    l4 = tf.nn.relu(tf.matmul(l3, w4) + biases)
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    biases = tf.get_variable('biases5', [12], initializer=tf.constant_initializer(0.0))
    pyx = tf.matmul(l4, w_o) + biases
    return pyx

result = img_group.get_data()
trX = np.array(result['train_images'])
trY = np.array(result['train_labels'])
teX = np.array(result['test_images'])
teY = np.array(result['test_labels'])

trX = trX.reshape(-1, width, height, 3)
teX = teX.reshape(-1, width, height, 3)

X = tf.placeholder('float', [None, width, height, 3])    # img shape = (40, 40, 3)
Y = tf.placeholder('float', [None, 12])                  # label = 12

w = init_weights([5, 5, 3, 32])
w2 = init_weights([5, 5, 32, 64])
w3 = init_weights([5, 5, 64, 128])
w4 = init_weights([128 * 5 * 5, 625])
w_o = init_weights([625, 12])

p_keep_conv = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')

py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.009, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    for i in xrange(50):
        training_batch = zip(xrange(0, len(trX), batch_size), 
                             xrange(batch_size, len(trX), batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], 
                                          p_keep_conv:0.8, p_keep_hidden:0.5})
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        localtime = time.asctime(time.localtime(time.time()))
        
        print (i, np.mean(np.argmax(teY[test_indices], axis=1) == 
                          sess.run(predict_op, feed_dict={X:teX[test_indices],
                                                          p_keep_conv:1.0,
                                                          p_keep_hidden:1.0})), localtime)
        

(0, 0.0546875, 'Sat May 14 14:02:08 2016')
(1, 0.05078125, 'Sat May 14 14:02:53 2016')
(2, 0.1328125, 'Sat May 14 14:03:37 2016')
(3, 0.125, 'Sat May 14 14:04:21 2016')
(4, 0.16015625, 'Sat May 14 14:05:06 2016')
(5, 0.10546875, 'Sat May 14 14:05:50 2016')
(6, 0.1171875, 'Sat May 14 14:06:36 2016')
(7, 0.13671875, 'Sat May 14 14:07:22 2016')
(8, 0.12890625, 'Sat May 14 14:08:07 2016')
(9, 0.1171875, 'Sat May 14 14:08:52 2016')
(10, 0.1484375, 'Sat May 14 14:09:38 2016')
(11, 0.17578125, 'Sat May 14 14:10:23 2016')
(12, 0.12890625, 'Sat May 14 14:11:09 2016')
(13, 0.14453125, 'Sat May 14 14:11:54 2016')
(14, 0.14453125, 'Sat May 14 14:12:41 2016')
(15, 0.09375, 'Sat May 14 14:13:26 2016')
(16, 0.14453125, 'Sat May 14 14:14:12 2016')
(17, 0.1171875, 'Sat May 14 14:14:57 2016')
(18, 0.1171875, 'Sat May 14 14:15:43 2016')
(19, 0.12890625, 'Sat May 14 14:16:30 2016')
(20, 0.125, 'Sat May 14 14:17:15 2016')
(21, 0.1171875, 'Sat May 14 14:18:00 2016')
(22, 0.12109375, 'Sat May 14 14:18:45 201

In [1]:
import img_group
import tensorflow as tf
import numpy as np
import time

batch_size = 128
test_size = 256
width = 40
height = 40

def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name = name)

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    
    # l1a shape = (?, 40, 40, 32)
    # l1 shpae = (?, 20, 20, 32)
    with tf.name_scope('layer1'):
        biases = tf.get_variable('biases1', [32], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(X, w, strides=[1,1,1,1], padding='SAME')
        bias = tf.nn.bias_add(conv1, biases)
        l1a = tf.nn.relu(bias)
        l1 = tf.nn.max_pool(l1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        l1 = tf.nn.dropout(l1, p_keep_conv)
    
    # l2a shape = (?, 20, 20, 64)
    # l2 shpae = (?, 10, 10, 64)
    with tf.name_scope('layer2'):
        biases = tf.get_variable('biases2', [64], initializer=tf.constant_initializer(0.1))
        conv2 = tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding='SAME')
        bias = tf.nn.bias_add(conv2, biases)
        l2a = tf.nn.relu(bias)
        l2 = tf.nn.max_pool(l2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        l2 = tf.nn.dropout(l2, p_keep_conv)
    
    # l3a shape = (?, 10, 10, 128)
    # l3 shpae = (?, 5, 5, 128)
    # l3 reshape to (?, 128*5*5)
    with tf.name_scope('layer3'):
        biases = tf.get_variable('biases3', [128], initializer=tf.constant_initializer(0.1))
        conv3 = tf.nn.conv2d(l2, w3, strides=[1,1,1,1], padding='SAME')
        bias = tf.nn.bias_add(conv3, biases)
        l3a = tf.nn.relu(bias)
        l3 = tf.nn.max_pool(l3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
        l3 = tf.nn.dropout(l3, p_keep_conv)
    
    with tf.name_scope('layer4'):
        biases = tf.get_variable('biases4', [625], initializer=tf.constant_initializer(0.1))
        l4 = tf.nn.relu(tf.matmul(l3, w4) + biases)
        l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    with tf.name_scope('layer5'):
        biases = tf.get_variable('biases5', [12], initializer=tf.constant_initializer(0.0))
        pyx = tf.matmul(l4, w_o) + biases
        return pyx

result = img_group.get_data()
trX = np.array(result['train_images'])
trY = np.array(result['train_labels'])
teX = np.array(result['test_images'])
teY = np.array(result['test_labels'])

trX = trX.reshape(-1, width, height, 3)
teX = teX.reshape(-1, width, height, 3)

X = tf.placeholder('float', [None, width, height, 3], name = 'X')    # img shape = (40, 40, 3)
Y = tf.placeholder('float', [None, 12], name = 'Y')                  # label = 12

w = init_weights([5, 5, 3, 32], 'w')
w2 = init_weights([5, 5, 32, 64], 'w2')
w3 = init_weights([5, 5, 64, 128], 'w3')
w4 = init_weights([128 * 5 * 5, 625], 'w4')
w_o = init_weights([625, 12], 'w_o')

# Add histogram summaries for weights
tf.histogram_summary('w_summ', w)
tf.histogram_summary('w2_summ', w2)
tf.histogram_summary('w3_summ', w3)
tf.histogram_summary('w4_summ', w4)
tf.histogram_summary('w_o_summ', w_o)

p_keep_conv = tf.placeholder('float', name = 'p_keep_conv')
p_keep_hidden = tf.placeholder('float', name = 'p_keep_hidden')

py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.03, 0.9).minimize(cost)
    tf.scalar_summary('cost', cost)
    
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, 'float'))    
    tf.scalar_summary('accuracy', acc_op)
    
with tf.Session() as sess:
    
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter('./logs/nn_logs', sess.graph)
    merged = tf.merge_all_summaries()
    
    tf.initialize_all_variables().run()
    
    for i in xrange(5):
        training_batch = zip(xrange(0, len(trX), batch_size), 
                             xrange(batch_size, len(trX), batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], 
                                          p_keep_conv:0.8, p_keep_hidden:0.5})
        
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        localtime = time.asctime(time.localtime(time.time()))
        
        summary, acc = sess.run([merged, acc_op], feed_dict={X:teX[test_indices], 
                                                             Y:teY[test_indices], 
                                                             p_keep_conv: 1.0, 
                                                             p_keep_hidden: 1.0})
        print (i, acc, localtime)


(0, 0.171875, 'Mon May 16 02:25:18 2016')
(1, 0.1015625, 'Mon May 16 02:26:07 2016')
(2, 0.15625, 'Mon May 16 02:26:57 2016')
(3, 0.09765625, 'Mon May 16 02:27:47 2016')
(4, 0.15625, 'Mon May 16 02:28:34 2016')
